In [ ]:
import tkinter as tk
from tkinter import messagebox, Menu, scrolledtext
import subprocess
import os
import random
import sys

class KSMGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("KAMA System Package Manager (KSM) GUI")

        # Create a menu bar
        self.menu_bar = Menu(root)
        root.config(menu=self.menu_bar)

        # Create a menu for switching to CLI
        self.file_menu = Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="File", menu=self.file_menu)
        self.file_menu.add_command(label="Switch to CLI", command=self.open_cli)

        # Example GUI elements (placeholders)
        self.url_label = tk.Label(root, text="URL:")
        self.url_label.pack(pady=5)
        self.url_entry = tk.Entry(root)
        self.url_entry.pack(pady=5)
        self.download_button = tk.Button(root, text="Download", command=self.download_file)
        self.download_button.pack(pady=20)

        # Log box
        self.log_box = scrolledtext.ScrolledText(root, height=10, width=50, state='normal')
        self.log_box.pack(pady=10)

        # Check for ManagerAttack
        self.check_manager_attack()

    def check_manager_attack(self):
        if random.random() < 0.10:  # 30% chance
            self.log("MANAGER ATTACK: exiting for safety.")
            messagebox.showerror("Error", "MANAGER ATTACK: exiting for safety.")
            self.root.after(1000, sys.exit)  # Exit after 1 second to allow message to be displayed

    def open_cli(self):
        # This method will open a new Command Prompt window and run the ksm_cli script
        try:
            # Get the path of the current working directory
            script_dir = os.getcwd()
            # Run the CLI script in a new Command Prompt window
            subprocess.Popen(['start', 'cmd', '/k', 'py -3.12', 'ksm_cli.ipy'], shell=True, cwd=script_dir)
        except Exception as e:
            self.log(f"Failed to open CLI: {e}")
            messagebox.showerror("Error", f"Failed to open CLI: {e}")

    def download_file(self):
        # Example function for downloading a file
        url = self.url_entry.get()
        if not url:
            self.log("URL cannot be empty.")
            messagebox.showerror("Error", "URL cannot be empty.")
            return
        
        try:
            # Run the KSM CLI download command in the same directory
            subprocess.run(['py -3.12', 'ksm_cli.ipy', 'download', url], check=True)
            self.log("File downloaded successfully.")
            messagebox.showinfo("Success", "File downloaded successfully.")
        except subprocess.CalledProcessError as e:
            self.log(f"Failed to download file: {e}")
            messagebox.showerror("Error", f"Failed to download file: {e}")

    def log(self, message):
        # Append message to the log box
        self.log_box.configure(state='normal')
        self.log_box.insert(tk.END, message + '\n')
        self.log_box.configure(state='disabled')
        self.log_box.yview(tk.END)  # Auto-scroll to the end of the log

if __name__ == "__main__":
    root = tk.Tk()
    app = KSMGUI(root)
    root.mainloop()


: 

In [8]:
import tkinter as tk
from tkinter import scrolledtext
import os
import requests
from urllib.parse import urlparse
import zipfile

# Define command functions
def cmd_print(parameters):
    return parameters

def cmd_list(parameters):
    return "\n".join(os.listdir('.'))

def cmd_exit(parameters):
    root.quit()

def cmd_kwd(parameters):
    url = parameters.strip()
    try:
        response = requests.get(url)
        filename = os.path.basename(urlparse(url).path)
        with open(filename, 'wb') as file:
            file.write(response.content)
        return f"Downloaded {filename}"
    except Exception as e:
        return f"Error downloading file: {e}"

def cmd_kwg(parameters):
    repo_url = parameters.strip()
    try:
        # Extract repo name from URL
        repo_name = os.path.basename(urlparse(repo_url).path).replace('.git', '')
        
        # Create a zip URL by appending `/archive/refs/heads/main.zip` (for GitHub)
        zip_url = f"{repo_url.replace('.git', '')}/archive/refs/heads/main.zip"
        
        # Download the zip file
        response = requests.get(zip_url)
        zip_filename = f"{repo_name}.zip"
        with open(zip_filename, 'wb') as file:
            file.write(response.content)
        
        # Extract the zip file
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(repo_name)
        
        return f"Downloaded and extracted repository into {repo_name}"
    except Exception as e:
        return f"Error with repository download: {e}"

def cmd_echo(parameters):
    return parameters

def cmd_pwd(parameters):
    return os.getcwd()

def cmd_cd(parameters):
    try:
        os.chdir(parameters.strip())
        return f"Changed directory to {os.getcwd()}"
    except FileNotFoundError:
        return "Directory not found"

def cmd_mkdir(parameters):
    try:
        os.makedirs(parameters.strip())
        return f"Directory created: {parameters.strip()}"
    except Exception as e:
        return f"Error creating directory: {e}"

def cmd_rmdir(parameters):
    try:
        os.rmdir(parameters.strip())
        return f"Directory removed: {parameters.strip()}"
    except Exception as e:
        return f"Error removing directory: {e}"

def cmd_touch(parameters):
    try:
        with open(parameters.strip(), 'w') as file:
            pass
        return f"File created: {parameters.strip()}"
    except Exception as e:
        return f"Error creating file: {e}"

def cmd_rm(parameters):
    try:
        os.remove(parameters.strip())
        return f"File removed: {parameters.strip()}"
    except Exception as e:
        return f"Error removing file: {e}"

def cmd_cat(parameters):
    try:
        with open(parameters.strip(), 'r') as file:
            return file.read()
    except Exception as e:
        return f"Error reading file: {e}"

def cmd_copy(parameters):
    src, dest = parameters.split()
    try:
        with open(src, 'rb') as fsrc:
            with open(dest, 'wb') as fdest:
                fdest.write(fsrc.read())
        return f"File copied from {src} to {dest}"
    except Exception as e:
        return f"Error copying file: {e}"

def cmd_move(parameters):
    src, dest = parameters.split()
    try:
        os.rename(src, dest)
        return f"File moved from {src} to {dest}"
    except Exception as e:
        return f"Error moving file: {e}"

def cmd_ls(parameters):
    return "\n".join(os.listdir(parameters.strip()))

def cmd_chmod(parameters):
    mode, filename = parameters.split()
    try:
        os.chmod(filename, int(mode, 8))
        return f"Permissions changed for {filename}"
    except Exception as e:
        return f"Error changing permissions: {e}"

def cmd_head(parameters):
    filename, num_lines = parameters.split()
    num_lines = int(num_lines)
    try:
        with open(filename, 'r') as file:
            lines = [next(file) for _ in range(num_lines)]
        return ''.join(lines)
    except Exception as e:
        return f"Error reading file: {e}"

def cmd_tail(parameters):
    filename, num_lines = parameters.split()
    num_lines = int(num_lines)
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()[-num_lines:]
        return ''.join(lines)
    except Exception as e:
        return f"Error reading file: {e}"

def cmd_find(parameters):
    filename = parameters.strip()
    for root, dirs, files in os.walk('.'):
        if filename in files:
            return os.path.join(root, filename)
    return "File not found"

def cmd_grep(parameters):
    search_str, filename = parameters.split(' ', 1)
    try:
        with open(filename, 'r') as file:
            return '\n'.join(line for line in file if search_str in line)
    except Exception as e:
        return f"Error reading file: {e}"

def cmd_diff(parameters):
    file1, file2 = parameters.split()
    try:
        with open(file1, 'r') as f1, open(file2, 'r') as f2:
            diff = [line for line in f1 if line not in f2]
        return ''.join(diff)
    except Exception as e:
        return f"Error comparing files: {e}"

# Command mapping
commands = {
    'print': cmd_print,
    'list': cmd_list,
    'exit': cmd_exit,
    'kwd': cmd_kwd,
    'kwg': cmd_kwg,
    'echo': cmd_echo,
    'pwd': cmd_pwd,
    'cd': cmd_cd,
    'mkdir': cmd_mkdir,
    'rmdir': cmd_rmdir,
    'touch': cmd_touch,
    'rm': cmd_rm,
    'cat': cmd_cat,
    'copy': cmd_copy,
    'move': cmd_move,
    'ls': cmd_ls,
    'chmod': cmd_chmod,
    'head': cmd_head,
    'tail': cmd_tail,
    'find': cmd_find,
    'grep': cmd_grep,
    'diff': cmd_diff
}

def parse_block(block):
    try:
        # Extract block type and content
        if '[' not in block or ']' not in block:
            return "Invalid block format."

        block_type = block.split('[')[0].strip('*')
        content = block.split('[')[1].split(']')[0]
        
        if '[' in content:
            command, parameters = content.split('[', 1)
        else:
            command = content
            parameters = ''

        command = command.strip()
        parameters = parameters.strip()

        # Process commands based on block type
        if block_type == 'strblk' and command in commands:
            return commands[command](parameters)
        elif block_type == 'fileblk' and command in commands:
            return commands[command](parameters)
        else:
            return f"Unknown or invalid block type: {block_type}"
    except (IndexError, ValueError) as e:
        return f"Error processing block: {e}"

def parse_kxx_file(filename):
    blocks = []
    with open(filename, 'r') as file:
        content = file.read()
        while '**' in content:
            start_idx = content.index('**')
            end_idx = content.find(']', start_idx) + 1
            if end_idx == -1:
                break
            block = content[start_idx:end_idx]
            content = content[end_idx:]
            blocks.append(block)
    return blocks

def process_block(block):
    output = parse_block(block)
    return output

def execute_command(command):
    try:
        # Direct command handling
        if command in commands:
            return commands[command]('')
        elif command.startswith('kwd '):
            return commands['kwd'](command[4:])
        elif command.startswith('kwg '):
            return commands['kwg'](command[4:])
        else:
            return f"Unknown command: {command}"
    except Exception as e:
        return f"Error executing command: {e}"

def process_input():
    user_input = command_entry.get().strip()
    if user_input:
        if user_input.startswith('**'):
            output = process_block(user_input)
        else:
            output = execute_command(user_input)
        output_text.config(state=tk.NORMAL)
        output_text.insert(tk.END, f"> {user_input}\n{output}\n")
        output_text.config(state=tk.DISABLED)
        command_entry.delete(0, tk.END)

def run_script():
    file_path = script_entry.get().strip()
    if file_path.endswith('.kxx'):
        if os.path.isfile(file_path):
            blocks = parse_kxx_file(file_path)
            for block in blocks:
                output = process_block(block)
                output_text.config(state=tk.NORMAL)
                output_text.insert(tk.END, f"{output}\n")
                output_text.config(state=tk.DISABLED)
        else:
            output_text.config(state=tk.NORMAL)
            output_text.insert(tk.END, f"File not found: {file_path}\n")
            output_text.config(state=tk.DISABLED)

# Tkinter setup
root = tk.Tk()
root.title("Kindvs Shell")

command_entry = tk.Entry(root, width=50)
command_entry.pack(pady=5)

run_button = tk.Button(root, text="Run Command", command=process_input)
run_button.pack(pady=5)

script_entry = tk.Entry(root, width=50)
script_entry.pack(pady=5)

run_script_button = tk.Button(root, text="Run Script", command=run_script)
run_script_button.pack(pady=5)

output_text = scrolledtext.ScrolledText(root, width=80, height=20, state=tk.DISABLED)
output_text.pack(pady=5)

# Start Tkinter event loop
root.mainloop()


: 